In [10]:
import pandas as pd
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models, tools
from biogeme.expressions import Beta, Variable, bioDraws, MonteCarlo, log, Power, exp, Derive, RandomVariable
import biogeme.distributions as dist
import scipy.stats as st
import time

Data description

The dataset `apollo_swissRouteChoiceData` comes from an actual SP survey of public transport route choice conducted in
Switzerland (Axhausen et al., 2008). A set of 388 people were faced with 9 choices each between two public transport routes, both using train (leading to 3'492 observations in the data). The two alternatives are described on the basis of travel time, travel cost, headway (time between subsequent trains/busses) and the number of interchanges. For each individual, the dataset
additionally contains information on income, car availability in the household, and whether the
journey was made for commuting, shopping, business or leisure.

See slide 79 Apollo Documentation for more info on Mixtures

Preparing the data

In [11]:
# Read the CSV file
df = pd.read_csv(r"C:\Users\alexi\Desktop\GITS\Semester-project\Data\apollo_swissRouteChoiceData.csv")
df

,ID,choice,tt1,tc1,hw1,ch1,tt2,tc2,hw2,ch2,hh_inc_abs,car_availability,commute,shopping,business,leisure
0,2439,2,58,7,30,1,50,8,30,0,50000,1,1,0,0,0
1,2439,1,30,8,60,0,41,7,15,2,50000,1,1,0,0,0
2,2439,1,41,7,30,0,34,8,15,2,50000,1,1,0,0,0
3,2439,1,44,10,60,1,52,9,60,2,50000,1,1,0,0,0
4,2439,2,43,9,60,0,34,10,30,0,50000,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3487,84525,1,22,4,15,0,24,3,15,0,70000,0,1,0,0,0
3488,84525,1,22,3,15,0,20,4,30,1,70000,0,1,0,0,0
3489,84525,1,20,3,15,2,16,4,15,2,70000,0,1,0,0,0
3490,84525,2,20,4,60,2,15,5,30,2,70000,0,1,0,0,0


In [12]:
#fill nAn values with 0 (same as in Apollo)
df = df.fillna(0) 

#creating biogeme database

database_MMNL = db.Database('MMNL_Preference_Model', df)
globals().update(database_MMNL.variables) #transform all columns into variables

Defining the model parameters

In [13]:
mu_log_b_tt = Beta('mu_log_b_tt', -3, None, None, 0)
sigma_log_b_tt = Beta('sigma_log_b_tt', -0.01, None, None, 0)
mu_log_b_tc = Beta('mu_log_b_tc', -3, None, None, 0)
sigma_log_b_tc = Beta('sigma_log_b_tc', -0.01, None, None, 0)
mu_log_b_hw = Beta('mu_log_b_hw', -3, None, None, 0)
sigma_log_b_hw = Beta('sigma_log_b_hw', -0.01, None, None, 0)
mu_log_b_ch = Beta('mu_log_b_tt', -3, None, None, 0)
sigma_log_b_ch = Beta('sigma_log_b_ch', -0.01, None, None, 0)

#no fixed parameters



Generate draws from Random Distribution (Halton Draws)

In [14]:
b_tt = -exp(mu_log_b_tt + sigma_log_b_tt*bioDraws('b_tt', 'NORMAL_HALTON5'))
b_tc = -exp(mu_log_b_tc + sigma_log_b_tc*bioDraws('b_tc', 'NORMAL_HALTON5'))
b_hw = -exp(mu_log_b_hw + sigma_log_b_hw*bioDraws('b_hw', 'NORMAL_HALTON5'))
b_ch = -exp(mu_log_b_ch + sigma_log_b_ch*bioDraws('b_ch', 'NORMAL_HALTON5'))



Defining the Model

In [15]:
#Availabilities not provided for 'apollo_choiceAnalysis', so full availability is assumed.

V_1 = b_tt * tt1 + b_tc * tc1 + b_hw * hw1 + b_ch * ch1
V_2 = b_tt * tt2 + b_tc * tc2 + b_hw * hw2 + b_ch * ch2


V_MMNL = {1: V_1, 2: V_2}


Estimating the model

In [16]:
# Define the model
logprob_MMNL = log(MonteCarlo(models.loglogit(V_MMNL, None, choice)))

#call cpu/ram function

# Estimate the model
biogeme_MMNL = bio.BIOGEME(database_MMNL, logprob_MMNL)
biogeme_MMNL.modelName = 'MMNL_Model'
biogeme_MMNL.generateHtml = True  # Disable HTML file generation
biogeme_MMNL.generatePickle = False  # Disable PICKLE file generation
biogeme_MMNL.save_iterations = False  # Disable ITER file 

start_time = time.time() #mesure time of estimation

results_model_MMNL = biogeme_MMNL.estimate()

end_time = time.time()
elapsed_time = end_time - start_time

# Output + time taken
print(results_model_MMNL.getEstimatedParameters())

print(f"Elapsed Time: {elapsed_time} seconds")

Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


: 

In [ ]:
# Retrieve the general statistics from the results
general_stats_model_RP = results_model_RP.getGeneralStatistics()
print(results_model_RP.printGeneralStatistics())